# practice 4. Logistic Regression 

데이터 셋 링크: https://www.kaggle.com/datasets/jsphyg/weather-dataset-rattle-package?select=weatherAUS.csv  
참고자료: https://www.kaggle.com/code/prashant111/logistic-regression-classifier-tutorial

이 데이터 집합에는 호주 전역의 여러 위치에서 약 10년간의 일일 기상 관측 데이터가 포함되어 있습니다. 내일 비가 올지 여부를 나타낸 RainTomorrow 여부를 예측하는 logistic regression 모델을 학습시키고, 성능을 측정하고, 결과를 해석해보세요.

- Date : 날짜
- Location : 위치
- MinTemp : 최저기온
- MaxTemp : 최고기온
- Rainfall : 강수량
- Evaporation : 증발
- Shnshine : 하루 동안 해가 떠있는 시간
- WindGustDir : 풍향
- WindGustSpped : 풍속
- WindDir9am : 오전 9시 풍향
- WindDir3pm : 오후 3시 풍향
- WindSpeed9am : 오전 9시 풍속
- WindSpeed3pm : 오후 3시 풍속
- Humidity9am : 오전 9시 습도
- Humidity3pm : 오후 3시 습도
- Pressure9am : 오전 9시 해수면 기압
- Pressure3pm : 오후 3시 해수면 기압
- Cloud9am : 오전 9시 구름
- Cloud3pm : 오후 3시 구름
- Temp9am : 오전 9시 기온
- Temp3pm : 오후 3시 기온
- RainToday : 당일 강수 여부

In [335]:
from utils import load_weather_dataset

x_train, x_test, y_train, y_test = load_weather_dataset()

In [336]:
train = pd.concat([x_train,y_train],axis=1)
test = pd.concat([x_test,y_test],axis=1)

In [337]:
train = train.dropna(subset = ['RainTomorrow'])
y_train = train['RainTomorrow']
x_train = train.drop('RainTomorrow',axis=1)

test = test.dropna(subset = ['RainTomorrow'])
y_test = test['RainTomorrow']
x_test = test.drop('RainTomorrow',axis=1)

In [ ]:
x_train.columns

In [ ]:
x_train.info()

In [ ]:
x_train.isnull().sum()

In [338]:
import pandas as pd

x_train['Date'] = pd.to_datetime(x_train['Date'])

x_train['Year'] = x_train['Date'].dt.year
x_train['Month'] = x_train['Date'].dt.month
x_train['Day'] = x_train['Date'].dt.day

x_train.drop('Date',axis=1,inplace=True)

In [339]:
x_test['Date'] = pd.to_datetime(x_test['Date'])

x_test['Year'] = x_test['Date'].dt.year
x_test['Month'] = x_test['Date'].dt.month
x_test['Day'] = x_test['Date'].dt.day

x_test.drop('Date',axis=1,inplace=True)

In [340]:
continuous = list(x_train.select_dtypes(include=('float','int')).columns)
categorical = list(x_train.select_dtypes(include='object').columns)

In [ ]:
x_train[continuous].isnull().sum()

In [341]:
for i in continuous:
    x_train[i] = x_train[i].fillna(x_train.groupby(['Location','Month'])[i].transform('mean'))
    x_train[i] = x_train[i].fillna(x_train.groupby(['Location'])[i].transform('mean'))
    x_train[i] = x_train[i].fillna(x_train.groupby(['Month'])[i].transform('mean'))
    x_test[i] = x_test[i].fillna(x_test.groupby(['Location','Month'])[i].transform('mean'))
    x_test[i] = x_test[i].fillna(x_test.groupby(['Location'])[i].transform('mean'))
    x_test[i] = x_test[i].fillna(x_test.groupby(['Month'])[i].transform('mean'))


In [ ]:
x_train[continuous].isnull().sum()

In [ ]:
x_test[continuous].isnull().sum()

===========================================================================================================

In [342]:
import numpy as np
import math

In [ ]:
x_train[categorical].isnull().sum()

In [239]:
wind_dict = x_train.groupby(["Location", "Month"])['WindGustDir'].agg(pd.Series.mode).to_dict()

In [343]:
for i in categorical:
    wind_dict = x_train.groupby(["Location", "Month"])[i].agg(pd.Series.mode).to_dict()
    for k, v in wind_dict.items():
        if isinstance(v, np.ndarray):
            city,month = k
            result = x_train[x_train['Month'] == month][i].mode().iloc[0]
            if len(v) == 0:
                wind_dict[k] = result
            elif len(v) >= 2:
                wind_dict[k] = v[0]
    
    def most_common_wind(x):
        wind = x[i]
        if isinstance(wind, str):
            return wind
        if not math.isnan(wind): 
            return wind
        location = x["Location"]
        month = x["Month"]
        wind = wind_dict[(location, month)]
        return wind            
    
    x_train[i] = x_train.apply(most_common_wind, axis=1)

In [344]:
for i in categorical:
    wind_dict = x_test.groupby(["Location", "Month"])[i].agg(pd.Series.mode).to_dict()
    for k, v in wind_dict.items():
        if isinstance(v, np.ndarray):
            city,month = k
            result = x_test[x_test['Month'] == month][i].mode().iloc[0]
            if len(v) == 0:
                wind_dict[k] = result
            elif len(v) >= 2:
                wind_dict[k] = v[0]
    
    def most_common_wind(x):
        wind = x[i]
        if isinstance(wind, str):
            return wind
        if not math.isnan(wind): 
            return wind
        location = x["Location"]
        month = x["Month"]
        wind = wind_dict[(location, month)]
        return wind            
    
    x_test[i] = x_test.apply(most_common_wind, axis=1)

In [299]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

In [306]:
for i in categorical:
    x_train[i] = label_encoder.fit_transform(x_train[i])

In [308]:
for i in categorical:
    x_test[i] = label_encoder.fit_transform(x_test[i])

In [345]:
for i in categorical:
    onehot = pd.get_dummies(x_train[i],prefix=i).astype(int)
    x_train = pd.concat([x_train,onehot],axis=1)
    x_train.drop([i],axis=1,inplace=True)

In [346]:
for i in categorical:
    onehot = pd.get_dummies(x_test[i],prefix=i).astype(int)
    x_test = pd.concat([x_test,onehot],axis=1)
    x_test.drop([i],axis=1,inplace=True)

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns

plt.figure(figsize=(15,13))

for i,variable in enumerate(continuous):
    if i == 9:
        break
    plt.subplot(3,3,i+1)
    sns.histplot(data = x_train, x= variable)

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns

plt.figure(figsize=(15,13))

for i,variable in enumerate(continuous):
    if i == 9:
        break
    plt.subplot(3,3,i+1)
    sns.boxplot(data = x_train, x= variable)

mintemp,maxtemp,windgustspeed,windspeed9am,windspeed3pm,humidity9am,humidity3pm,pressure9am,pressure3pm,temp9am,temp3pm = standard scaling
rainfall,evaporation,sunshine,cloud9am,cloud3pm = minmax scaling

In [ ]:
plt.figure(figsize=(15,13))

for i,variable in enumerate(continuous):
    if i < 9 or i > 15:
        continue
    plt.subplot(3,3,i-8)
    sns.histplot(data = x_train, x= variable)

In [ ]:
plt.figure(figsize=(15,13))

for i,variable in enumerate(continuous):
    if i < 9 or i > 15:
        continue
    plt.subplot(3,3,i-8)
    sns.boxplot(data = x_train, x= variable)

In [ ]:
mintemp,maxtemp,windgustspeed,windspeed9am,windspeed3pm,humidity9am,humidity3pm,pressure9am,pressure3pm,temp9am,temp3pm = standard scaling
rainfall,evaporation,sunshine,cloud9am,cloud3pm = minmax scaling

In [347]:
from sklearn.preprocessing import RobustScaler,MinMaxScaler,StandardScaler

robust_scaler = RobustScaler()
minmax_scaler = MinMaxScaler()
standard_scaler = StandardScaler()

In [357]:
x_train_scaler = robust_scaler.fit_transform(x_train)
x_test_scaler = robust_scaler.transform(x_test)

In [358]:
x_train_scaler = pd.DataFrame(
    x_train_scaler,
    index=x_train.index,
    columns=x_train.columns
)

In [359]:
x_test_scaler = pd.DataFrame(
    x_test_scaler,
    index=x_test.index,
    columns=x_test.columns
)

In [351]:
y_train = y_train.map({'Yes': 1, 'No': 0})
y_test = y_test.map({'Yes': 1, 'No': 0})

In [360]:
import statsmodels.api as sm

x_constant_train = sm.add_constant(x_train_scaler)
model = sm.Logit(y_train,x_constant_train)
result = model.fit()

Optimization terminated successfully.
         Current function value: 0.349028
         Iterations 7


In [361]:
x_constant_test = sm.add_constant(x_test_scaler)

In [362]:
from sklearn.metrics import accuracy_score

y_train_pred = result.predict(x_constant_train) > 0.5
y_test_pred = result.predict(x_constant_test) > 0.5

In [363]:
print('train accuracy', round(accuracy_score(y_train,y_train_pred),4))

train accuracy 0.8498


In [364]:
print('train accuracy', round(accuracy_score(y_test,y_test_pred),4))

train accuracy 0.8468


In [368]:
np.exp(result.params).sort_values(ascending=False)

Humidity3pm             6.028608
Pressure9am             3.063113
WindGustSpeed           2.200441
Location_Adelaide       1.941512
Location_Perth          1.827479
                          ...   
Location_Townsville     0.420266
Location_Wollongong     0.404133
Location_MountGinini    0.352783
Pressure3pm             0.190383
const                   0.186210
Length: 119, dtype: float64